In [1]:
# Install required packages
!pip install litellm transformers datasets

# Import necessary libraries
import os
import subprocess
import sys
import tempfile
from datasets import load_dataset


print("Dependencies installed and libraries imported successfully.")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dependencies installed and libraries imported successfully.


In [2]:
import json

# Read the JSON file
with open('/Users/isavita/git/advent-of-code/train.json', 'r') as f:
    data = json.load(f)

In [3]:
import os

# Add asdf paths to the PATH environment variable
os.environ['PATH'] = "/opt/homebrew/bin:" + os.environ['PATH']
os.environ['PATH'] = os.path.expanduser("~/.asdf/bin:~/.asdf/shims") + ":" + os.environ['PATH']

# Verify the Elixir executable
!which elixir
!elixir --version


/Users/isavita/.asdf/shims/elixir
Erlang/OTP 26 [erts-14.1] [source] [64-bit] [smp:8:8] [ds:8:8:10] [async-threads:1] [jit]

Elixir 1.15.7 (compiled with Erlang/OTP 26)


In [101]:
class Constants:
    @staticmethod
    def file_ext():
        return "tcl"
    
    @staticmethod
    def lang():
        return "tcl"
    
    @staticmethod
    def compiler():
        return "/opt/homebrew/bin/tclsh"
    
    @staticmethod
    def runtime():
        return "/opt/homebrew/bin/tclsh" #"/opt/homebrew/bin/kotlinc main.kt -include-runtime -d main.jar && java -jar main.jar" # return "/opt/homebrew/opt/llvm@14/bin/clang  -fobjc-arc -framework Foundation main.m -o main && ./main"

print("Go constants class defined.")

Go constants class defined.


In [102]:
import tempfile
import subprocess
import os

def test_go_execution():
    # Hello World program in Go
    code = '''
puts "Hello, World!"
'''
    
    # Create a temporary file
    with tempfile.NamedTemporaryFile(suffix=f'.{Constants.file_ext()}', delete=False, mode='w') as temp_file:
        temp_file.write(code)
        temp_file_path = temp_file.name
    
    try:
        # Run the program
        result = subprocess.run(f"{Constants.runtime()} {temp_file_path}", shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("Program executed successfully.")
            print("Output:", result.stdout.strip())
        else:
            print("Error executing program:")
            print(result.stderr)
    finally:
        # Clean up the temporary file
        os.unlink(temp_file_path)

# Run the test
test_go_execution()

Program executed successfully.
Output: Hello, World!


Python(17030) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [103]:
import litellm
import warnings
import os
from openai import OpenAI

# Suppress the specific warning
warnings.filterwarnings("ignore", message="Field .* has conflict with protected namespace .*")

class LLMWrapper:
    def __init__(self, model_name, api_base=None):
        self.model_name = model_name
        self.api_base = api_base
        self.temperature = 0.0  # Hardcoded temperature as specified
        self.is_fine_tuned = model_name.startswith("ft:")
        if self.is_fine_tuned:
            self.client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY_FT"))

    def generate(self, system_prompt, user_prompt):
        if self.is_fine_tuned:
            return self._generate_with_openai(system_prompt, user_prompt)
        else:
            return self._generate_with_litellm(system_prompt, user_prompt)

    def _generate_with_litellm(self, system_prompt, user_prompt):
        try:
            completion_params = {
                "model": self.model_name,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                # "temperature": self.temperature,
                "timeout": 2*100,
            }
            
            if self.api_base:
                completion_params["api_base"] = self.api_base

            response = litellm.completion(**completion_params)
            content = response.choices[0].message.content
            if self.model_name == "ollama/marco-o1":
                return content[(content.index("<Output>") + 8):content.index("</Output>")]
            else:
                return content
        except Exception as e:
            print(f"Error in LLM API call: {str(e)}")
            return None

    def _generate_with_openai(self, system_prompt, user_prompt):
        try:
            if not self.client.api_key:
                raise ValueError("OPENAI_API_KEY environment variable not set")

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=self.temperature
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in OpenAI API call: {str(e)}")
            return None

# Test the LLMWrapper with the local Ollama model
def test_llm_wrapper():
    wrapper = LLMWrapper("ollama/llama3:8b-instruct-q5_K_M", api_base="http://localhost:11434")
    
    system_prompt = "You are a helpful assistant."
    user_prompt = "Tell me a short joke about programming."
    
    print("Testing with local Ollama model:")
    response = wrapper.generate(system_prompt, user_prompt)
    if response:
        print("Local LLM Response:")
        print(response)
    else:
        print("Failed to get a response from the local LLM.")

# Run the test
# test_llm_wrapper()

In [104]:
import threading
import time
import queue

class TimeoutRunner:
    @staticmethod
    def run_with_timeout(func, args=(), kwargs={}, timeout_duration=10):
        result_queue = queue.Queue()

        def wrapper():
            try:
                result = func(*args, **kwargs)
                result_queue.put(result)
            except Exception as e:
                result_queue.put(e)

        thread = threading.Thread(target=wrapper)
        thread.daemon = True
        thread.start()
        
        try:
            return result_queue.get(timeout=timeout_duration)
        except queue.Empty:
            return None

# Test the TimeoutRunner
def test_timeout_runner():
    def quick_function():
        return "Quick function completed"

    def slow_function():
        time.sleep(5)
        return "Slow function completed"

    runner = TimeoutRunner()

    print("Testing quick function (should complete):")
    result = runner.run_with_timeout(quick_function, timeout_duration=2)
    print(result)

    print("\nTesting slow function (should timeout):")
    result = runner.run_with_timeout(slow_function, timeout_duration=2)
    print(result)

# Run the test
test_timeout_runner()

Testing quick function (should complete):
Quick function completed

Testing slow function (should timeout):
None


In [105]:
import os
import tempfile
import subprocess

class EvaluateCode:
    def __init__(self, code, input_data):
        self.code = code
        self.input_data = input_data

    def run(self):
        with tempfile.TemporaryDirectory() as temp_dir:
            # Save code to a file using the extension from Constants
            code_file = os.path.join(temp_dir, f"main.{Constants.file_ext()}")
            with open(code_file, 'w') as f:
                f.write(self.code)

            # Save input to a file
            input_file = os.path.join(temp_dir, "input.txt")
            with open(input_file, 'w') as f:
                f.write(self.input_data)

            # Directly run the file
            run_command = f"{Constants.runtime()} {code_file}"

            # Run the command in the temporary directory
            run_result = subprocess.run(
                run_command,
                shell=True,
                capture_output=True,
                text=True,
                cwd=temp_dir  # Set the working directory to the temporary directory
            )

            if run_result.returncode != 0:
                return f"Runtime error: {run_result.stderr}"

            return run_result.stdout

# Test the EvaluateCode class
def test_evaluate_code():
    code = '''
print("Enter a number:")
n = io.read("*n")
print("The number is: " .. n)
'''
    input_data = "43"

    evaluator = EvaluateCode(code, input_data)
    result = evaluator.run()
    print("Evaluation result:")
    print(result)

# Run the test
test_evaluate_code()

Evaluation result:
Runtime error: invalid command name "print("Enter"
    while executing
"print("Enter a number:")"
    (file "/var/folders/46/6lz6t86n4n10lwsyj3pyzd6h0000gn/T/tmp_dy2z5tf/main.tcl" line 2)



Python(17031) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [106]:
import re
import os
import signal
import time
from pathlib import Path

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Execution timed out")

class ChallengeSolver:
    def __init__(self, model_name, llm_wrapper, language='julia'):
        self.model_name = model_name
        self.llm_wrapper = llm_wrapper
        self.language = language

    def extract_code(self, input_text):
        pattern = rf"```(?:{self.language}|r)?(.*?)```"
        matches = re.findall(pattern, input_text, re.DOTALL)
        return matches[0] if matches else input_text

    def valid_solution(self, result, answer):
        special_cases = [
            ".##..####.###..#..#.###..####.###....##.###...###.",
            " ##  #### ###  #  # ###  #### ###    ## ###   ### ",
            "#....#..#....#.....###..######....##....#....#....##....######",
            "#    #  #    #     ###  ######    ##    #    #    ##    ######",
            "3.465154e+06", "3.465154e+6",
            "####.###..####.#..#.###..\n#....#..#....#.#..#.#..#.",
            "#### ###  #### #  # ###  \n#    #  #    # #  # #  # ",
            ".#....###....##.#..#.####.#..#.#....#..#.",
            " #    ###    ## #  # #### #  # #    #  #",
            " █    ███    ██ █  █ ████ █  █ █    █  █",
            "#..#.#..#.#..#.#..#.#..#.#..#.#..#....#",
            "#  # #  # #  # #  # #  # #  # #  #    #",
            "###..###..###...##..###...##...##..####.",
            "###  ###  ###   ##  ###   ##   ##  #### "
        ]
        return answer in result or any(case in result for case in special_cases)

    def solve(self, challenge_data, use_go_solution=False):
        if use_go_solution:
            system_prompt = f"""
            You are an expert programmer that writes simple, concise code and no comments or explanation.
            Given this golang program write a {self.language} version that it reads its input from a file "input.txt" and solve the following task.
            The program should print the answer.
            Ensuring that the translated program includes a proper `main` entry point and optimizes it if possible.

            Python Solution:
            {challenge_data['solution']}
            """
            user_prompt = f"Convert the above Python solution to {self.language} and optimize it if possible."
        else:
            system_prompt = f"""
            Write a {self.language} program that reads input from a file called input.txt and prints the output to standard output.
            Focus on writing clean, efficient code that demonstrates your programming skills by concisely solving the challenge.
            {"**NOTE:** DO NOT forget to add at the top of the code #!/usr/bin/awk" if self.language == 'awk' else ""}{"**NOTE:** DO NOT forget to add at the top of the code #lang racket" if self.language == 'racket' else ""}

            Ensuring that the translated program includes a proper `main` entry point and optimizes it if possible.
            Coding challenge:
            """
            user_prompt = challenge_data['task']

        start_time = time.time()
        timeout = 2*120  # 40 seconds timeout

        # Set the signal handler and a 40-second alarm
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        try:
            max_attempts = 1
            for attempt in range(max_attempts):
                if time.time() - start_time > timeout:
                    raise TimeoutException("Execution timed out")

                response = self.llm_wrapper.generate(system_prompt, user_prompt)
                # print(response)
                code = self.extract_code(response)
                # print(code)

                evaluator = EvaluateCode(code, challenge_data['input'])
                result = evaluator.run()

                if self.valid_solution(result, challenge_data['answer']):
                    self.save_solution(challenge_data, code)
                    return True, result

            return False, "Failed to find a valid solution after multiple attempts."

        except TimeoutException:
            return False, "Execution timed out after 40 seconds."

        finally:
            # Cancel the alarm
            signal.alarm(0)

    def save_solution(self, challenge_data, code):
        home_dir = str(Path.home())
        file_path = f"{home_dir}/code/advent_generated/{challenge_data['name']}.{self.get_file_extension()}"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            f.write(code)

    def get_file_extension(self):
        extensions = {
            'go': 'go',
            'python': 'py',
            'awk': 'awk',
            'zig': 'zig',
            'ruby': 'rb',
            'lua': 'lua',
            'julia': 'jl',
            'php': 'php',
            'elixir': 'exs',
            'perl': 'pl',
            'crystal': 'cr',
            'dart': 'dart',
            'nim': 'nim',
            'clojure': 'clj',
            'scala': 'scala',
            'java': 'java',
            'kotlin': 'kt',
            'ocaml': 'ocaml',
            'groovy': 'groovy',
            'racket': 'rkt',
            'coffeescript': 'coffee',
            'R': 'R',
            'haskell': 'hs',
            'swift': 'swift',
            'objectivec': 'm',
            'pascal': 'pas',
            'javascript': 'js',
            'tcl': 'tcl',
            'wren': 'wren',
            'typescript': 'ts',
            'scheme': 'scm',
            'v': 'v',
            'bash': 'sh',
            'fortran90': 'f90',
            'csharp': 'cs',
            'fsharp': 'fs',
            'cpp': 'cpp',
            'c': 'c',
            'rust': 'rs',
            # Add more languages and their extensions as needed
        }
        return extensions.get(self.language, '')

# Example usage and test
def test_challenge_solver():
    class MockLLMWrapper:
        def generate(self, system_prompt, user_prompt):
            return '''Here's a Python solution for the problem:

```python
#!/usr/bin/env python3
with open('input.txt', 'r') as f:
    data = f.read().strip()

sum = 0
for i in range(len(data)):
    if data[i] == data[(i + 1) % len(data)]:
        sum += int(data[i])

print(sum)
```'''

    # # Sample challenge data
    # challenge_data = {
    #     "name": "day1_part1_2017",
    #     "task": "Find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.",
    #     "input": "1122",
    #     "answer": "3",
    #     "solution": "package main\n\nimport (\n\t\"fmt\"\n\t\"os\"\n\t\"strings\"\n)\n\nfunc main() {\n\tdata, _ := os.ReadFile(\"input.txt\")\n\tinput := strings.TrimSpace(string(data))\n\tsum := 0\n\tfor i := 0; i < len(input); i++ {\n\t\tif input[i] == input[(i+1)%len(input)] {\n\t\t\tsum += int(input[i] - '0')\n\t\t}\n\t}\n\tfmt.Println(sum)\n}",
    #     "solution_lang": "go",
    #     "year": 2017
    # }

    # solver = ChallengeSolver("gpt-3.5-turbo", MockLLMWrapper(), language='python')
    
    # print("Testing with task description:")
    # success, result = solver.solve(challenge_data, use_go_solution=False)
    # print(f"Solution found: {success}")
    # print(f"Result: {result}")

    # print("\nTesting with Go solution conversion:")
    # success, result = solver.solve(challenge_data, use_go_solution=True)
    # print(f"Solution found: {success}")
    # print(f"Result: {result}")

# Run the test
# test_challenge_solver()

In [109]:
import os
import time
from datasets import load_dataset
from pathlib import Path

def get_existing_solutions(language):
    home_dir = str(Path.home())
    solutions_dir = os.path.join(home_dir, "code", "advent_generated", "tcl")
    solutions = os.listdir(solutions_dir)
    return [os.path.splitext(solution)[0] for solution in solutions if solution.endswith(f".{language}")]

def run_max_times(func, max_attempts=1):
    for _ in range(max_attempts):
        success, result = func()
        if success:
            return True, result
    return False, "Failed after maximum attempts"

def main():
    target_language = 'tcl'  # Change this to 'awk' or any other language as needed

    with open('/Users/isavita/git/advent-of-code/train.json', 'r') as f:
        dataset = json.load(f)
    go_solutions = [example for example in dataset if example['solution_lang'] == 'python' and example['year'] != 2025]

    existing_solutions = get_existing_solutions('tcl')
    unsolved_challenges = [challenge for challenge in go_solutions if challenge['name'] not in existing_solutions]
    # "deepinfra/Qwen/Qwen2.5-Coder-32B-Instruct" "claude-3-5-haiku-20241022" "gemini/gemini-1.5-flash-latest" "openai/gpt-4.5-preview"
    # "mistral/codestral-latest" "groq/deepseek-r1-distill-llama-70b" "mistral/mistral-small-latest" "openai/o3-mini"
    # "gemini/gemini-2.0-flash-exp" "gemini/gemini-exp-1206" "gemini/gemini-2.0-flash-thinking-exp-01-21" "ollama/marco-o1"
    # "gemini/gemini-2.0-pro-exp" "together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo" "groq/deepseek-r1-distill-llama-70b"
    # "gemini/gemma-3-27b-it" "gemini/gemini-2.0-pro-exp-02-05"
    llm_wrapper = LLMWrapper("gemini/gemini-2.0-flash-exp")  # You can change this to your preferred model "ft:gpt-4o-mini-2024-07-18:personal:zig:9pcaR1MI" "gpt-4o-mini"
    solver = ChallengeSolver("gemini/gemini-2.0-flash-exp", llm_wrapper, language=target_language)

    total_challenges = len(unsolved_challenges)
    print(f"Total unsolved challenges: {total_challenges}")
    # unsolved_challenges.reverse()
    for i, challenge in enumerate(unsolved_challenges, 1):
        def solve_attempt():
            success, result = solver.solve(challenge, use_go_solution=False)
            print(result)
            if "Timeout" in result:
                print(f"Challenge {challenge['name']} timed out.")
                return False, result
            return success, result

        success, result = run_max_times(solve_attempt, max_attempts=3)

        if success:
            print(f"Solved {challenge['name']} successfully!")
        else:
            print(f"Failed to solve {challenge['name']}: {result}")

        progress = (i / total_challenges) * 100
        print(f"Progress: {progress:.1f}%")
        
        time.sleep(30)  # Sleep for 50 seconds between attempts

if __name__ == "__main__":
    main()

Total unsolved challenges: 149


Python(22772) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Failed to find a valid solution after multiple attempts.


Python(22773) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Failed to find a valid solution after multiple attempts.


...

Failed to find a valid solution after multiple attempts.


Python(24885) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Failed to find a valid solution after multiple attempts.


Python(24886) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Failed to find a valid solution after multiple attempts.
Failed to solve day24_part2_2024: Failed after maximum attempts
Progress: 100.0%


In [47]:
params = {
    "model": "ollama/marco-o1",
    "messages": [
        {"role": "system", "content": "You are a helpful Software Engineer"},
        {"role": "user", "content": "write me solution of fizzbuzz in Python"}
    ],
    "temperature": 0.0,
}
# resp = litellm.completion(**params)
# msg = resp.choices[0].message.content
# output = msg[(msg.index("<Output>") + 8):msg.index("</Output>")]